In [1]:
import cv2, torch, torchvision
from torch.utils.data import DataLoader

C:\Users\hoang\anaconda3\envs\ComputerVision\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import model
import torch.nn as nn
from torch.optim import Adam
from dataset import FreiHand


In [3]:
COLORMAP = {
    "thumb": {"ids": [0, 1, 2, 3, 4], "color": (0,255,0) #green
             },
    "index": {"ids": [0, 5, 6, 7, 8], "color": (0,255,255) #cyan
             },
    "middle": {"ids": [0, 9, 10, 11, 12], "color": (0,0,255) #blue
              },
    "ring": {"ids": [0, 13, 14, 15, 16], "color": (255,0,255) #pink
            },
    "little": {"ids": [0, 17, 18, 19, 20], "color": (255,0,0) #red
              },
}

In [5]:
config = {
    "data_dir": "Dataset/FreiHand",
    "n_sample": 1,
    "epochs": 1,
    "batch_size": 1,
    "shuffle":True,
    "batches_per_epoch": 50,
    "batches_per_epoch_val": 20,
    "learning_rate": 0.001,
    "decay": 0.0001,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}

In [6]:
module = model.Resnet(3,model.ResBottleneckBlock,[3,4,6,3], useBottleneck=True)
module=module.to(config["device"])

In [7]:
# Define the loss function with Classification Cross-Entropy loss and an optimizer with Adam optimizer
loss_fn = nn.MSELoss()
optimizer = Adam(module.parameters(), lr=0.001, weight_decay=0.0001)

In [8]:
train_set = FreiHand(config=config, set_type="train")
train_dl = DataLoader(train_set, batch_size=config["batch_size"], shuffle= True)

val_set = FreiHand(config=config, set_type="val")
val_dl = DataLoader(val_set, batch_size=config["batch_size"], shuffle= False)


In [ ]:
for _ in range(config["epochs"]):
    for i, data in enumerate(train_dl):
        optimizer.zero_grad()
        inputs= data["image"].to(config["device"])
        targets= data["heatmaps"].to(config["device"])
        yhat=module(inputs).to(torch.float64)

        loss= loss_fn(yhat, targets)
        print(loss)
        loss.backward()
        optimizer.step()

tensor(0.2368, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.2193, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.2014, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.1853, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.1740, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.1418, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.1250, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.1094, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0922, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0794, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0699, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0624, device='cuda:0', 

tensor(0.0230, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0136, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0117, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0139, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0142, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0124, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0140, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0271, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0137, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0190, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0153, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
tensor(0.0177, device='cuda:0', 

In [ ]:
bla=train_set[0]['keypoints'].detach().cpu().numpy()

In [ ]:
bla

In [ ]:
bla=bla.reshape(2,21)*32

In [ ]:
from skimage.transform import resize
bla2 = resize(train_set[0]['image'].detach().cpu().numpy().transpose([2, 1, 0]), (32, 32))

In [ ]:
bla2

In [ ]:
import matplotlib.pyplot as plt


plt.subplot(1,2,2)
plt.imshow(train_set[0]['heatmaps'][17])
plt.subplot(1,2,1)
plt.imshow(bla2)
plt.scatter(bla[0][17],bla[1][17])

In [ ]:
output

In [ ]:
from PIL import Image
from torchvision import transforms
bla=Image.open("test.jpg")

In [ ]:
bla.numpy

In [ ]:
image_transform = transforms.Compose([
				transforms.Resize(128),
				transforms.ToTensor(),
				])

In [ ]:
bla1= image_transform(bla)

In [ ]:
bla1.shape